## Evaluating the predictive accuracy of physics emulators without hyperparameter fine tuning

In [1]:
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from duqling_interface import DuqlingInterface

from plot_performance import heatmap

In [2]:
from sklearn.cross_decomposition      import PLSRegression
from sklearn.linear_model             import LassoLars, ElasticNet
from sklearn.ensemble                 import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm                      import SVR
from sklearn.gaussian_process         import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from sklearn.pipeline                 import Pipeline
from sklearn.preprocessing            import StandardScaler

models = [
    PLSRegression(n_components=1),
    LassoLars(random_state=42),
    ElasticNet(max_iter=10_000, random_state=42),
    ExtraTreesRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    SVR(kernel='rbf'),
    Pipeline([
        ('scale', StandardScaler()),
        ('gpr',   GaussianProcessRegressor(
            kernel = ConstantKernel(1.0, (1e-2, 1e2))
                    * Matern()
                    + WhiteKernel(1e-3, (1e-6, 1e1)),
            n_restarts_optimizer = 3,
            random_state = 42,
            normalize_y = True
        ))
    ])
]

In [4]:
duq = DuqlingInterface()

univariate_funcs = duq.list_functions(response_type='uni').fname

As Kelin pointed out, latin hypercube sampling will produce distinct distributions of data based on the random seed.

In [ ]:
duqling_funcs = duq.list_functions(response_type='uni').fname
model_names = [model.__class__.__name__ for model in models]

df_mse = pd.DataFrame(columns=duqling_funcs, index=model_names)

In [ ]:
for fname in tqdm(duqling_funcs):
    if fname == 'cube3_rotate':
        continue
    X_trn, y_trn = duq.generate_data(fname, 1000, seed=41)
    X_tst, y_tst = duq.generate_data(fname, 1000, seed=42)
    for model in models:
        model.fit(X_trn, y_trn)
        y_pred = model.predict(X_tst)
        df_mse.at[model.__class__.__name__, fname] = mean_squared_error(y_pred, y_tst)

  0%|          | 0/19 [00:00<?, ?it/s]

/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py:348: UserWarning: y residual is constant at iteration 0
  warnings.warn(f"y residual is constant at iteration {k}")
/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/reidmorris/Desktop/duqling/duqling_ven

In [ ]:
df_rmse = df_mse.map(np.sqrt)

/var/folders/mg/16bsy2y90s3fzg_d4crt_vgh0000gn/T/ipykernel_32760/1379462558.py:1: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [1]:
fig1 = heatmap(df_rmse, "MSE")
# fig2 = heatmap((df_rmse/df_std).drop(cols_to_drop, axis=1), "Test RMSE / \u03C3")
# fig3 = heatmap((df_rmse/df_std)[(df_rmse/df_std).drop(cols_to_drop, axis=1)>1].drop(cols_to_drop, axis=1), "(Test RMSE / \u03C3) > 1")

NameError: name 'heatmap' is not defined